In [0]:
# !pip install tensorflow==1.14.

In [0]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random

In [0]:
%tensorflow_version 2.x
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

Using TensorFlow backend.


In [ ]:
# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

# def read_dataset(path):
#     X = []
#     y = []
    
#     image_paths_list = glob(os.path.join(path, 'train', '*.jpg'))
#     print(path)
#     image_paths_sample = random.sample(image_paths_list, 100)

#     for image_path in image_paths_sample:
#         image_name = os.path.basename(image_path)
#         image_name_parts = image_name.split('.')
#         label = image_name_parts[0] if len(image_name_parts) == 3 else None

#         if label:
#             y.append(int(label == 'cat'))
            
#         x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
#         x = preprocess_input(x)
            
#         X.append(x)
            
#     return np.array(X), y

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [0]:
from random import shuffle
from tqdm import tqdm_notebook

def read_dataset(path, batch_size = 1000, n = 2):
    X = np.array([])
    y = []
    image_paths_list = glob(os.path.join(path, '*.jpg'))
    
    assert(batch_size <= len(image_paths_list))
    assert(n <= len(image_paths_list) / batch_size)
    
    shuffle(image_paths_list)
    bot, top = 0, batch_size
    
    for i in tqdm_notebook(range(n)):
        x_var = []
        y_var = []
        image_paths_sample = image_paths_list[bot:top]
        for image_path in tqdm_notebook(image_paths_sample, leave=False):
            image_name = os.path.basename(image_path)
            image_name_parts = image_name.split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None
            if label:
                y_var.append(int(label == 'cat'))

            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
            x_var.append(x)
        
        # x_var = np.expand_dims(x_var, axis=0)
        x_var = preprocess_input(np.array(x_var), mode=y_var)   

        if X.size == 0 and len(y) == 0:            
          X = x_var
          y = y_var
        else:
          X = np.vstack((X, x_var))
          y += y_var
                      
        bot = top
        top += batch_size
    return X, np.array(y)

In [0]:
# Используйте функцию read_dataset чтобы получить обучающую выборку

train_directory = '/content/drive/My Drive/data/train'

x_set, y_set = read_dataset(train_directory)

# Проверьте размерности загруженных данных
print(x_set.shape, y_set.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



(2000, 224, 224, 3) (2000,)


In [0]:
# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге
x_ftrs, y_ftrs, x_lbls, y_lbls = train_test_split(x_set, y_set, train_size=0.7, test_size=0.3)
print(x_ftrs.shape, y_ftrs.shape, x_lbls.shape, y_lbls.shape)

(1400, 224, 224, 3) (600, 224, 224, 3) (1400,) (600,)


In [0]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

In [0]:
# Используя объект model постройте эмбеддинги для обучающей выборки
# print(help(model))
embedding_trn = model.predict(x_ftrs)
embedding_tst = model.predict(y_ftrs)
print(embedding_trn)

[[2.2301683e-07 1.7656786e-06 2.6048033e-07 ... 2.4963077e-07
  5.5113407e-05 2.0251640e-03]
 [1.2981116e-05 3.5813446e-05 4.0455111e-06 ... 9.3189856e-06
  4.8495445e-04 3.7866279e-03]
 [3.1075695e-05 1.0209326e-04 8.9089917e-06 ... 2.4864146e-06
  5.7080388e-04 1.1593970e-03]
 ...
 [1.0330824e-08 1.8947433e-09 1.4473631e-08 ... 5.5323263e-10
  3.4690316e-08 6.1315552e-07]
 [3.2319008e-06 1.9962538e-06 2.3002643e-07 ... 5.6752384e-07
  1.3676177e-05 1.3021965e-05]
 [6.1102969e-06 6.0270959e-06 9.2171746e-07 ... 2.0465388e-06
  1.7196630e-04 3.6166646e-05]]


Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [0]:
from xgboost import XGBClassifier
# Получите предсказания на тестовом и на обучающем множестве
cls = XGBClassifier()
cls.fit(embedding_trn, x_lbls)
predicted = cls.predict(embedding_tst)

In [0]:
# С помощью функции accuracy_score оцените результаты
print(accuracy_score(y_lbls, predicted))

0.9933333333333333


In [0]:
hyper_cls = XGBClassifier(n_jobs=-1, n_estimators=150)

e_set = [(embedding_trn, x_lbls), (embedding_tst, y_lbls)]

hyper_cls.fit(embedding_trn, x_lbls, eval_set=e_set, verbose=0)
hyper_predicted = hyper_cls.predict(embedding_tst)
print(accuracy_score(y_lbls, hyper_predicted))

0.9916666666666667


Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

In [0]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from sklearn.model_selection import cross_val_score

def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params, n_jobs=-1)
    current_score = cross_val_score(clf, embedding_trn, x_lbls, cv=3).mean()
    print(current_score, params)
    return -current_score
 
 
simple_space_xgb = {
            'learning_rate': hp.quniform('learning_rate', 0.001, 0.1, 0.001),
            'max_depth':  hp.choice('max_depth', np.arange(3, 10, dtype=int)),
            'n_estimators': hp.choice('n_estimators', np.arange(130, 250, dtype=int)),
}
 
best = fmin(fn=hyperopt_xgb_score, space=simple_space_xgb, algo=tpe.suggest, max_evals=10)
print("\nbest: ", best)

0.9764270156509912
{'learning_rate': 0.08, 'max_depth': 6, 'n_estimators': 218}
0.9757132397766157
{'learning_rate': 0.047, 'max_depth': 6, 'n_estimators': 204}
0.9764270156509912
{'learning_rate': 0.074, 'max_depth': 5, 'n_estimators': 193}
0.9771407915253666
{'learning_rate': 0.073, 'max_depth': 8, 'n_estimators': 180}
0.9764270156509912
{'learning_rate': 0.073, 'max_depth': 7, 'n_estimators': 160}
0.9771407915253666
{'learning_rate': 0.069, 'max_depth': 9, 'n_estimators': 159}
0.9649943479979046
{'learning_rate': 0.004, 'max_depth': 8, 'n_estimators': 181}
0.9757132397766157
{'learning_rate': 0.042, 'max_depth': 6, 'n_estimators': 181}
0.9735688487377195
{'learning_rate': 0.042, 'max_depth': 5, 'n_estimators': 142}
0.9728550728633442
{'learning_rate': 0.017, 'max_depth': 3, 'n_estimators': 166}
100%|██████████| 10/10 [06:14<00:00, 37.44s/it, best loss: -0.9771407915253666]

best:  {'learning_rate': 0.073, 'max_depth': 5, 'n_estimators': 50}


In [0]:
optimized_model = XGBClassifier(**best, n_jobs=-1)
optimized_model.fit(embedding_trn, x_lbls)
opt_predicted = optimized_model.predict(embedding_tst)
print(optimized_model)
print(accuracy_score(y_lbls, opt_predicted))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.073, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
0.9833333333333333


Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки

In [0]:
!pip install lxml
!pip install bs4

In [0]:
from bs4 import BeautifulSoup
import requests
import re

url = "https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a"
req = requests.get(url)
soup = BeautifulSoup(req.text, "lxml")

pattern = re.compile(r"(\d+): (\"|\')(\D+)(\'|\")")
model_class = dict()
for i in range(1, 1000):
  fs = soup.find("td", {"id": f"file-imagenet1000_clsidx_to_labels-txt-LC{i}"})
  value_of_tag = pattern.findall(fs.text)
  model_class[int(value_of_tag[0][0])] = value_of_tag[0][2]

feature_imp = cls.feature_importances_
for i in range(len(feature_imp)):
  if feature_imp[i] > 1e-3:
    print(i, model_class[i])

1 goldfish, Carassius auratus
30 bullfrog, Rana catesbeiana
42 agama
63 Indian cobra, Naja naja
65 sea snake
74 garden spider, Aranea diademata
87 African grey, African gray, Psittacus erithacus
107 jellyfish
136 European gallinule, Porphyrio porphyrio
155 Shih-Tzu
158 toy terrier
159 Rhodesian ridgeback
163 bloodhound, sleuthhound
164 bluetick
166 Walker hound, Walker foxhound
167 English foxhound
168 redbone
170 Irish wolfhound
171 Italian greyhound
174 Norwegian elkhound, elkhound
177 Scottish deerhound, deerhound
182 Border terrier
185 Norfolk terrier
194 Dandie Dinmont, Dandie Dinmont terrier
200 Tibetan terrier, chrysanthemum dog
204 Lhasa, Lhasa apso
206 curly-coated retriever
207 golden retriever
208 Labrador retriever
209 Chesapeake Bay retriever
219 cocker spaniel, English cocker spaniel, cocker
225 malinois
234 Rottweiler
240 Appenzeller
244 Tibetan mastiff
245 French bulldog
249 malamute, malemute, Alaskan malamute
250 Siberian husky
255 Leonberg
256 Newfoundland, Newfoundl